## Packages et données

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! pip install rasterio  -q -q -q
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q

In [3]:
!pip install scikit-image

In [4]:
import sys
sys.path.append('../src')
from classes.data.satellite_image import SatelliteImage
from utils.utils import *
from utils.plot_utils import *
from utils.image_utils import *
import utils.mappings as mapps

In [5]:
from train_pipeline_utils.download_data import load_satellite_data
from classes.data.satellite_image import SatelliteImage

In [6]:
import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import re
from pyproj import Transformer
from datetime import date
from scipy.ndimage import label
import time
from tqdm import tqdm
import os

In [7]:
load_satellite_data(2022, "972", "PLEIADES")

In [57]:
load_satellite_data(2022, "973", "PLEIADES")

In [9]:
load_satellite_data(2020, "976", "PLEIADES")

In [10]:
load_satellite_data(2017, "976", "PLEIADES")

In [115]:
load_satellite_data(2022, "973", "PLEIADES")

## Images en t et t+n avec des différences

En martinique :
(14.635338, -61.038345) plateau médico-technique construit en 2018 (côté droit du triangle). 
Essayons de retrouver l'image qui le contient.

In [12]:
#Nouvelle image 1
filename_1 = find_image_of_point([14.635338, -61.038345], '../data/PLEIADES/2022/MARTINIQUE', coord_gps = True)
date_1 = date.fromisoformat('2022-01-01')

In [13]:
image_1 = SatelliteImage.from_raster(
        filename_1,
        date = date_1, 
        n_bands = 3,
        dep = "972"
    )

In [9]:
image_1.plot([0,1,2]) 

In [ ]:
#image 2
filename_2 = find_image_different_years(2017, None, filename_1)
date_2 = date.fromisoformat('2017-01-01')

In [ ]:
image_2 = SatelliteImage.from_raster(
        filename_2,
        date = date_2, 
        n_bands = 3,
        dep = "972"
    )

In [89]:
image_2.plot([0,1,2])

On a bien un changement à droite du triangle. Il ne reste plus qu'a avoir la bande infrarouge pour faire les masques puis les différences.

On va chercher en guyane : (-12.755601, 45.229945) coordonnées d'un quartier

In [14]:
filename_5 = find_image_of_point([-12.755601, 45.229945], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_5 = date.fromisoformat('2020-01-01')
print(filename_5)

In [15]:
image_5 = SatelliteImage.from_raster(
        filename_5,
        date = date_5, 
        n_bands = 3,
        dep = "976"
    )

In [18]:
image_5.plot([0,1,2])

In [16]:
filename_6 = find_image_different_years(2017, image_5)
date_6 = date.fromisoformat('2020-01-01')

In [17]:
image_6 = SatelliteImage.from_raster(
        filename_6,
        date = date_6, 
        n_bands = 3,
        dep = "976"
    )

In [150]:
image_6.plot([0,1,2])

Dans la moitié haute de l'image à droite, il y a plusieurs bâtiments qui ont été construits.

Pas de RIL pour ces deux années en mayotte ...

In [19]:
filename_7 = find_image_of_point([-12.753436, 45.219888], '../data/PLEIADES/2020/MAYOTTE', coord_gps = True)
date_7 = date.fromisoformat('2020-01-01')
print(filename_7)

In [20]:
image_7 = SatelliteImage.from_raster(
        filename_7,
        date = date_7, 
        n_bands = 3,
        dep = "976"
    )

In [295]:
image_7.plot([0,1,2])

In [21]:
#filename_8 = find_image_of_point([-12.753436, 45.219888], '../data/PLEIADES/2017/MAYOTTE', coord_gps = True)
filename_8 = find_image_different_years(2017, image_7)
date_8 = date.fromisoformat('2017-01-01')

In [23]:
image_8 = SatelliteImage.from_raster(
        filename_8,
        date = date_8, 
        n_bands = 3,
        dep = "976"
    )

In [24]:
image_8.plot([0,1,2])

### Image en uint16 sans image en t+n pour tester les masques sur des images en uint16

In [75]:
filename = '../data/PLEIADES/2022/GUYANE/ORT_2022072050325085_0355_0544_U22N_16Bits.jp2'
image = SatelliteImage.from_raster(
        filename,
        date = None, 
        n_bands = 4,
        dep = '973'
    )

In [36]:
image.plot([0,1,2])

### Exporter une SatelliteImage en png (pour LabelStudio)

In [374]:
from PIL import Image
import numpy as np
import io

# Créer un tableau avec des données d'exemple
data = image_5.array
data = data.transpose(1,2,0)


# Convertir le tableau en image
image = Image.fromarray(data)

# Sauvegarder l'image en tant que fichier PNG
image.save("image5.png")

# Enregistrer l'image en tant qu'objet bytes pour téléchargement
img_bytes = io.BytesIO()
image.save(img_bytes, format='PNG')
img_bytes.seek(0)

# Envoyer l'image pour téléchargement
from IPython.display import FileLink
display(FileLink("image5.png"))

### Import de masques depuis LabelStudio

In [39]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "3",
            name_output = "mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits"
        )

mask  = np.load("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy")
plt.imshow(np.transpose(mask), cmap="gray", interpolation="nearest")

np.save("../export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy",np.transpose(mask))

mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0524_8590_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.jp2
mc cp export/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0524_8590_U38S_8Bits.npy


In [30]:
from utils.gestion_label_studio import create_mask_from_label_studio_export

create_mask_from_label_studio_export(
            num_task = "1",
            name_output = "mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit"
        )


mc cp s3/projet-slums-detection/Donnees/PLEIADES/MAYOTTE/2020/MOSA/ORT_2020052526670967_0523_8591_U38S_8Bits.jp2 s3/projet-slums-detection/donnees-test/segmentation/images/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bits.jp2

mc cp export/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy s3/projet-slums-detection/donnees-test/segmentation/masks/mayotte-ORT_2020052526670967_0523_8591_U38S_8Bit.npy


## Essais de masques sur seuils pixels

Mask pour les images en uint8

In [11]:
def mask_inversion(image, threshold = 110):
    img = image.array.copy()
    img = img[:3,:,:]
    img = (img * 255).astype(np.uint8)
    img = img.transpose(1,2,0)

    shape = img.shape[0:2]

    grayscale = np.mean(img, axis=2)
    
    #quantile = 0.1
    #threshold = np.quantile(grayscale, quantile) 
    black = np.ones(shape, dtype=float)
    white = np.zeros(shape, dtype=float)

    # Creation of the mask : all grayscaled prixels below the threshold \
    # will be black and all the grayscaled prixels above the threshold \
    # will be white.

    mask = np.where(grayscale > threshold, white, black)
    
    return(mask)

    # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
    # ax1.imshow(grayscale, cmap="gray")
    # ax1.set_title("niveaux de gris")
    # ax1.axis('off')
    # ax2.imshow(mask, cmap="gray")
    # ax2.set_title("mask")
    # ax2.axis('off')
    # plt.show()

In [73]:
mask1 = mask_inversion(image_7, 110)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
ax1.imshow(image_7.array.transpose(1,2,0))
ax1.set_title("image")
ax1.axis('off')
ax2.imshow(mask1, cmap="gray")
ax2.set_title("mask")
ax2.axis('off')
plt.show()

In [33]:
mask2 = mask_inversion(image_8, 95)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
ax1.imshow(image_8.array.transpose(1,2,0))
ax1.set_title("image")
ax1.axis('off')
ax2.imshow(mask2, cmap="gray")
ax2.set_title("mask")
ax2.axis('off')
plt.show()

Mask pour les images en uint16

In [120]:
def mask_inversion2(image, threshold = 220):
    img = image.array.copy()
    img = img[:3,:,:]
    max = np.max(img)
    img = (img/max)*255
    img = img.astype(np.uint8)
    img = img*255
    img = img.transpose(1,2,0)
    grayscale = np.mean(img, axis=2)

    shape = img.shape[0:2]

    grayscale = np.mean(img, axis=2)

    #quantile = 0.1
    #threshold = np.quantile(grayscale, quantile)
    black = np.ones(shape, dtype=float)
    white = np.zeros(shape, dtype=float)

    # Creation of the mask : all the grayscaled prixels below the threshold \
    # will be black and all the grayscaled prixels above the threshold \
    # will be white.

    mask = np.where(grayscale > threshold, white, black)
    
    return(mask)

In [133]:
mask = mask_inversion2(image, 220)

image_copy = image.copy()
image_copy.normalize()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
ax1.imshow(np.transpose(image_copy.array, (1, 2, 0))[:, :, [0,1,2]])
ax1.set_title("image")
ax1.axis('off')
ax2.imshow(mask, cmap = "gray")
ax2.set_title("mask")
ax2.axis('off')
plt.show()

### Tests du masque pixel sur les patchs du train data martinique 2022

essais des seuils sur 10 images

In [56]:
file_path = "../train_data-PLEIADES-972-2022"

list_name = os.listdir(file_path + "/images")
list_path = [ file_path + "/images/" + name for name in list_name]

for path in tqdm(list_path[:10]):
    try:
        big_satellite_image = SatelliteImage.from_raster(
            file_path=path, dep=None, date=None, n_bands=3
        )
        
    except RasterioIOError:
        continue

    else:
        mask = mask_inversion(big_satellite_image, 142)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
        ax1.imshow(big_satellite_image.array.transpose(1,2,0))
        ax1.set_title("image de base")
        ax1.axis('off')
        ax2.imshow(mask, cmap="gray")
        ax2.set_title("mask")
        ax2.axis('off')
        plt.show()

Pour enregistrer les masques dans un dossier :

In [36]:
file_path = "../train_data-PLEIADES-972-2022"
output_directory_name = '../masksimplepleiades/train/2022/MARTINIQUE'

output_masks_path = output_directory_name

if os.path.exists(output_masks_path):
    print("fichiers déjà écrits")

if not os.path.exists(output_masks_path):
    os.makedirs(output_masks_path)


list_name = os.listdir(dir + "/images")
list_path = [ dir + "/images/" + name for name in liste]


for path, file_name in tqdm(zip(list_path, list_name), total=len(list_path), desc='Processing'):
    try:
        big_satellite_image = SatelliteImage.from_raster(
            file_path=path, dep=None, date=None, n_bands=3
        )
        
    except RasterioIOError:
        continue

    else:
        mask = mask_inversion(big_satellite_image, 142)
        file_name = file_name.split(".")[0]
        np.save(output_masks_path + "/" + file_name + ".npy", mask)

## Masques de différence

Lissage des masque en amont (pas obligatoire mais peut etre mieux)

In [34]:
from scipy.ndimage import uniform_filter

In [39]:
def lissage_mask(mask, neighborhood_size = 40, threshold = 0.5):
    # Taille du voisinage pour le lissage
    
    # Érosion suivie de dilatation (lissage)
    eroded_array = uniform_filter(mask, size=neighborhood_size, mode='constant', origin=0)
    smoothed_array = uniform_filter(eroded_array, size=neighborhood_size, mode='constant', origin=0)
    
    # Seuillage pour obtenir un masque binaire
    binary_mask = smoothed_array >= threshold
    binary_array = binary_mask.astype(np.uint8)

    return(binary_array)

In [70]:
mask_liss1 = lissage_mask(mask1, neighborhood_size = 15, threshold = 0.1)
mask_liss2 = lissage_mask(mask2, neighborhood_size = 15, threshold = 0.1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
ax1.imshow(mask_liss2, cmap = "gray")
ax1.set_title("lissage mask 2017")
ax1.axis('off')
ax2.imshow(mask_liss1, cmap = "gray")
ax2.set_title("lissage mask 2020")
ax2.axis('off')
plt.show()

In [36]:
def mask_diff(mask1, mask2):
    array1 = (mask1*255).astype(np.uint8)
    array2 = (mask2*255).astype(np.uint8)
    
    # Perform XOR operation on the arrays
    result = np.bitwise_xor(array1, array2)

    return(result)

In [71]:
diff_mask = mask_diff(mask_liss1, mask_liss2)

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.imshow(diff_mask, cmap = "gray")
ax.set_title("mask de diff avec bruits")
ax.axis('off')
plt.show()

Lissage de masque de diff

In [74]:
from scipy.ndimage import label

In [87]:
mask = (diff_mask*255).astype(np.uint8)
min_size = 3000
# Trouver les amas de pixels blancs correspondant à 5% ou plus de l'image
#threshold = 250  # seuil pour considérer un pixel comme "blanc"

labeled, num_features = label(mask)

region_sizes = np.bincount(labeled.flat)[:]

# Trier les labels de région en fonction de leur taille décroissante
sorted_labels = np.argsort(-region_sizes)

# Minimum size of the cluster
mask_filtered = np.zeros_like(labeled)

if num_features >= 1:
    for i in tqdm(range(1, num_features + 1)): # Display the progress bar
        if region_sizes[sorted_labels[i]] >= min_size:
            mask_filtered[labeled == sorted_labels[i]] = 1
        else:
            break

# Afficher l'image originale et l'image résultante côte à côte
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))
ax1.imshow(np.transpose(image_8.array, (1, 2, 0))[:, :, [0,1,2]])
ax1.set_title("image 2017")
ax1.axis('off')
ax2.imshow(np.transpose(image_7.array, (1, 2, 0))[:, :, [0,1,2]])
ax2.set_title("image 2020")
ax2.axis('off')
ax3.imshow(mask_filtered, cmap = "gray")
ax3.set_title("masque de différence filtré")
ax3.axis('off')
plt.show()

In [88]:
import geopandas as gpd
from shapely.geometry import Polygon
from rasterio.features import rasterize, shapes
import math

In [114]:
nchannel, height, width = (3, 2000, 2000)

# Create a list of polygons from the masked center clouds in order
# to obtain a GeoDataFrame from it
polygon_list_center = []
for shape in list(shapes(diff_mask)):
    polygon = Polygon(shape[0]["coordinates"][0])
    surface = polygon.area
    perimetre = polygon.length
    RP = perimetre/(2*np.pi)
    RS = math.sqrt(surface/np.pi)
    # if RP/RS > 2:
    #     continue
    if polygon.area > 0.85 * height * width:
        print("coucou")
        continue
    if polygon.area < 0.0007 * height * width:
        continue
    polygon_list_center.append(polygon)

result = gpd.GeoDataFrame(geometry=polygon_list_center)
print(result)


#Rasterize the geometries into a numpy array
if result.empty:
    rasterized = np.zeros((2000,2000))
else:
    rasterized = rasterize(
        result.geometry,
        out_shape=(2000,2000),
        fill=0,
        out=None,
        all_touched=True,
        default_value=1,
        dtype=None,
    )

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))
ax1.imshow(np.transpose(image_8.array, (1, 2, 0))[:, :, [0,1,2]])
ax1.set_title("image 2017")
ax1.axis('off')
ax2.imshow(np.transpose(image_7.array, (1, 2, 0))[:, :, [0,1,2]])
ax2.set_title("image 2020")
ax2.axis('off')
ax3.imshow(rasterized, cmap = "gray")
ax3.set_title("masque de différence filtré")
ax3.axis('off')
plt.show()

In [120]:
def mask_diff_tt1(image_t, image_t1):

    mask_t = mask_inversion(image_t, 110)
    mask_t1 = mask_inversion(image_t1, 110)

    mask_liss1 = lissage_mask(mask_t, neighborhood_size = 15, threshold = 0.1)
    mask_liss2 = lissage_mask(mask_t1, neighborhood_size = 15, threshold = 0.1)

    diff_mask = mask_diff(mask_liss1, mask_liss2)

    nchannel, height, width = (3, 2000, 2000)

    # Create a list of polygons from the masked center clouds in order
    # to obtain a GeoDataFrame from it
    polygon_list_center = []
    for shape in list(shapes(diff_mask)):
        polygon = Polygon(shape[0]["coordinates"][0])
        surface = polygon.area
        perimetre = polygon.length
        RP = perimetre/(2*np.pi)
        RS = math.sqrt(surface/np.pi)
        # if RP/RS > 2:
        #     continue
        if polygon.area > 0.85 * height * width:
            continue
        if polygon.area < 0.0007 * height * width:
            continue
        polygon_list_center.append(polygon)
    
    result = gpd.GeoDataFrame(geometry=polygon_list_center)
    
    
    #Rasterize the geometries into a numpy array
    if result.empty:
        rasterized = np.zeros((2000,2000))
    else:
        rasterized = rasterize(
            result.geometry,
            out_shape=(2000,2000),
            fill=0,
            out=None,
            all_touched=True,
            default_value=1,
            dtype=None,
        )

    return(rasterized)

In [121]:
rasterized = mask_diff_tt1(image_5, image_6)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 20))
ax1.imshow(np.transpose(image_6.array, (1, 2, 0))[:, :, [0,1,2]])
ax1.set_title("image 2017")
ax1.axis('off')
ax2.imshow(np.transpose(image_5.array, (1, 2, 0))[:, :, [0,1,2]])
ax2.set_title("image 2020")
ax2.axis('off')
ax3.imshow(rasterized, cmap = "gray")
ax3.set_title("masque de différence filtré")
ax3.axis('off')
plt.show()